In [53]:
!wget https://raw.githubusercontent.com/towardsai/tutorials/master/sentiment_analysis_tutorial/women_clothing_review.csv

--2021-01-29 21:01:15--  https://raw.githubusercontent.com/towardsai/tutorials/master/sentiment_analysis_tutorial/women_clothing_review.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8480236 (8.1M) [text/plain]
Saving to: ‘women_clothing_review.csv.5’

women_clothing_revi 100%[===================>]   8.09M  35.2MB/s    in 0.2s    

2021-01-29 21:01:15 (35.2 MB/s) - ‘women_clothing_review.csv.5’ saved [8480236/8480236]



In [54]:
import pandas as pd
import numpy as np
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow as tf


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
df = pd.read_csv('women_clothing_review.csv')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [56]:
df = df.drop(['Title', 'Positive Feedback Count', 'Unnamed: 0',], axis = 1)
df.dropna(inplace=True)
df['Polarity_Rating'] = df['Rating'].apply(lambda x: 'Positive' if x > 3 else ('Neutral' if x == 3 else 'Negative'))
df.head()

,Clothing ID,Age,Review Text,Rating,Recommended IND,Division Name,Department Name,Class Name,Polarity_Rating
0,767,33,Absolutely wonderful - silky and sexy and comf...,4,1,Initmates,Intimate,Intimates,Positive
1,1080,34,Love this dress! it's sooo pretty. i happene...,5,1,General,Dresses,Dresses,Positive
2,1077,60,I had such high hopes for this dress and reall...,3,0,General,Dresses,Dresses,Neutral
3,1049,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,General Petite,Bottoms,Pants,Positive
4,847,47,This shirt is very flattering to all due to th...,5,1,General,Tops,Blouses,Positive


In [57]:
df_Positive = df[df['Polarity_Rating'] == 'Positive'].sample(8000, replace=True )
df_Neutral = df[df['Polarity_Rating'] == 'Neutral'].sample(8000, replace=True )
df_Negative = df[df['Polarity_Rating'] == 'Negative'].sample(8000, replace=True )

df = pd.concat([df_Positive, df_Neutral, df_Negative], axis=0)
df.head()

,Clothing ID,Age,Review Text,Rating,Recommended IND,Division Name,Department Name,Class Name,Polarity_Rating
10014,1110,46,"Thin for this late in the season, but the colo...",5,1,General,Dresses,Dresses,Positive
13503,1094,36,I bought this in the orange color. might not b...,5,1,General,Dresses,Dresses,Positive
7185,860,60,"Love this top. looks great on, not uncomfortab...",5,1,General,Tops,Knits,Positive
21244,1126,27,"1st, this jacket fits true to size. 2nd, the s...",4,0,General,Jackets,Outerwear,Positive
589,823,37,This is a beautiful top. it extended longer th...,5,1,General,Tops,Blouses,Positive


In [58]:
def preprocess_test(text):
    stpword = stopwords.words('english')
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])

df['review'] = df['Review Text'].apply(preprocess_test)
df = df[['review', 'Polarity_Rating']]
df.head()


,review,Polarity_Rating
10014,Thin late season colors pretty im going layer ...,Positive
13503,bought orange color might color everyone color...,Positive
7185,Love top looks great uncomfortable right lengt...,Positive
21244,1st jacket fits true size 2nd shoulders tight ...,Positive
589,beautiful top extended longer hoped petite siz...,Positive


In [59]:
dummies = pd.get_dummies(df["Polarity_Rating"])
df.drop(['Polarity_Rating'], axis=1, inplace=True)
df = pd.concat([df, dummies ], axis=1)
df.head()


,review,Negative,Neutral,Positive
10014,Thin late season colors pretty im going layer ...,0,0,1
13503,bought orange color might color everyone color...,0,0,1
7185,Love top looks great uncomfortable right lengt...,0,0,1
21244,1st jacket fits true size 2nd shoulders tight ...,0,0,1
589,beautiful top extended longer hoped petite siz...,0,0,1


In [60]:
x = df['review'].values
y = df.drop('review', axis=1).values
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.30, random_state=42)
prediction_inputs = [xtest[:10], ytest[:10]]

In [61]:
vect = CountVectorizer()
xtrain = vect.fit_transform(xtrain)
xtest = vect.transform(xtest)
tfidf = TfidfTransformer()
xtrain = tfidf.fit_transform(xtrain)
xtest = tfidf.transform(xtest)
xtrain = xtrain.toarray()
xtest = xtest.toarray()

In [63]:
k = tf.keras
model = k.Sequential()

model.add(k.layers.Dense(units=1024,activation='relu'))
model.add(k.layers.Dropout(0.5))

model.add(k.layers.Dense(units=512,activation='relu'))
model.add(k.layers.Dropout(0.5))

model.add(k.layers.Dense(units=256,activation='relu'))
model.add(k.layers.Dropout(0.5))

model.add(k.layers.Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.fit(x=xtrain, y=ytrain, batch_size=256, epochs=5, validation_data=(xtest, ytest))

model_score = model.evaluate(xtest, ytest, batch_size=64, verbose=1)
print('Test accuracy:', model_score[1])

Epoch 1/5
66/66 [==============================] - 23s 335ms/step - loss: 0.9653 - accuracy: 0.4904 - val_loss: 0.4773 - val_accuracy: 0.8111
Epoch 2/5
66/66 [==============================] - 22s 327ms/step - loss: 0.3040 - accuracy: 0.8925 - val_loss: 0.2825 - val_accuracy: 0.9040
Epoch 3/5
66/66 [==============================] - 22s 329ms/step - loss: 0.0788 - accuracy: 0.9767 - val_loss: 0.2815 - val_accuracy: 0.9247
Epoch 4/5
66/66 [==============================] - 22s 330ms/step - loss: 0.0182 - accuracy: 0.9963 - val_loss: 0.3414 - val_accuracy: 0.9256
Epoch 5/5
113/113 [==============================] - 4s 38ms/step - loss: 0.3684 - accuracy: 0.9317
Test accuracy: 0.9316666722297668


In [64]:
preds = model.predict(xtest)
for pred in preds[:10]:
  print(np.where(pred == pred.max())[0])
print(prediction_inputs)

[2]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[1]
[0]
[array(['love top although wish ordered large rather medium 54 138 pounds 34d medium fits ever slightly snug across shoulders work though probably dry cleaned rather risk shrinkage washing home',
       'difficult find shirt doesnt expose half bra lift arm cute beyond arm hole issue overpriced going back',
       'loved aesthetic true hourglass shape frustrated didnt little give tried pull head fabric stiff seemed lace cotton entirely sure couldnt get past feeling material',
       'received mail today first slip included fabric totally sheer description says viscose lining assume slip picture looks like beige color second obviously sent store lost slip squished smallest possible retailer envelope fabric beautiful dress humongous like previous reviewers said ordered size smaller us',
       'Super itchy return',
       'tried instore intending buy trying shirts needed see would look jeans loved great medium blue color fit nicely 52 actual hips th